## 导包

In [2]:
import jieba
import pickle
import numpy as np

#### 将存的字典，tf值取出
这里data.pk 是[tf-idf.ipynb](https://github.com/JackKuo666/Similarity_matching_system/blob/b4d070b86b1c102a3de7a7ceab723e8c2c9d807a/jieba_tfidf/tf-idf.ipynb)中的练习部分可以自己完成，也可以用其他格式保存（csv,json等）；其中all_dick是{分词：序号}，idf_dict是{分词:idf值}

In [7]:

with open('data.pk', 'rb') as f:
    all_dick, idf_dict = pickle.load(f)

print([{i:all_dick[i]} for i in all_dick][:10])

print([{i:idf_dict[i]} for i in idf_dict][:10])


[{'首付': 0}, {'g9650': 1}, {'n1': 2}, {'黑金': 3}, {'l880': 4}, {'ds': 5}, {'mate8': 6}, {'夹': 7}, {'价': 8}, {'3gram': 9}]
[{'首付': 8.70631297394995}, {'g9650': 8.706252975149908}, {'n1': 8.706219224242849}, {'黑金': 8.706182537182249}, {'l880': 8.70631297394995}, {'ds': 8.706207973687352}, {'mate8': 8.70638796738806}, {'夹': 8.70638796738806}, {'价': 8.70631297394995}, {'3gram': 8.706237974887404}]


## 文件读取

#### 按行读取文件

In [44]:
# 按行读取文件
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8-sig') as f:
        fina_outlist = [line.strip() for line in f.readlines()]
    return fina_outlist

#### 按每行读取，每一行按空格切分为一个list，组成2维列表。

In [45]:
# 按每行读取，每一行按空格切分为一个list，组成2维列表。
def read_file2matrix(file_path):
    fina_outlist = []
    with open(file_path, 'r', encoding='utf-8-sig') as f:
        for line in f.readlines():
            outlist = [float(i) for i in line.strip().split(' ') if i != ' ']
            fina_outlist.append(outlist)
    return fina_outlist

## 分词

#### 加载自定义词典

In [46]:
jieba.load_userdict("userdict.txt")

#### 去停用词

In [47]:
# 将停用词读出放在stopwords这个列表中
filepath = r'stopwords.txt'
stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]

#### 搜索模式分词

In [48]:
def split_words(words):
    word_list = jieba.cut_for_search(words.lower().strip(),HMM=True)
    word_list = [i for i in word_list if i not in stopwords and i!=' '] 
    return word_list

## tfidf

#### 统计词频

In [49]:
# 统计词频，并返回字典
def make_word_freq(word_list):
    freword = {}
    for i in word_list:
        if str(i) in freword:
            freword[str(i)] += 1
        else:
            freword[str(i)] = 1
    return freword

#### 计算tfidf值

In [50]:
# 计算tfidf,组成tfidf矩阵
def make_tfidf(word_list,all_dick,idf_dict):
    length = len(word_list)
    word_list = [word for word in word_list if word in all_dick]
    word_freq = make_word_freq(word_list)
    w_dic = np.zeros(len(all_dick))
    for word in word_list:
        ind = all_dick[word]
        idf = idf_dict[word]
        w_dic[ind] = float(word_freq[word]/length)*float(idf)
    return w_dic

## 相似度计算

#### 余弦相似度

In [51]:
# 基于numpy的余弦相似性计算
def Cos_Distance(vector1, vector2):
    vec1 = np.array(vector1)
    vec2 = np.array(vector2)
    return float(np.sum(vec1 * vec2)) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

#### 计算相似度

In [52]:
# 计算相似度
def similarity_words(vec, vecs_list):
    Similarity_list = []
    for vec_i in vecs_list:
        Similarity = Cos_Distance(vec, vec_i)
        Similarity_list.append(Similarity)
    return Similarity_list

## 主函数

In [53]:
def main(words, file_path, readed_path):
    words_list = read_file(file_path)                    
    # 按行读取文本
    # ['Apple iPhone 8 Plus (A1864) 64GB 深空灰色 移动联通电信4G手机', '荣耀 畅玩7X 4GB+32GB 全网通4G全面屏手机 标配版 铂光金', 'Apple iPhone 8 (A1863) 64GB 深空灰色 移动联通电信4G手机', 'Apple iPhone 7 Plus (A1661) 128G 黑色 移动联通电信4G手机',  
    
    vecs_list = read_file2matrix(readed_path)
    # 按行读取tf-idf词袋 
    # [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.791469566521, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
    
    word_list = split_words(words)
    # 待对比语句读取并放在list里
    # ['apple', 'iphone', 'plus', 'a1864', '64gb', '深空', '灰色', '联通', '电信', '4g']
    
    vec = make_tfidf(word_list,all_dick,idf_dict)
    # 计算tfidf,组成tfidf矩阵
    """
      0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.79146957  0. 
    
    """
    
    similarity_lists = similarity_words(vec, vecs_list)
    # 计算相似度
    # [1.0000000000000002, 0.16116438715917517, 0.85811543720398442, 0.66742265521921251, 0.39477066819942941, 0.60302174693627475, 0.81818098341043122, 0.43081962853380523, 0.35533402523151619, 0.17407746930651016]
    
    sorted_res = sorted(enumerate(similarity_lists), key=lambda x: x[1])
    # 从小到大排序
    # [(84, 0.0), (87, 0.0), (92, 0.0), (119, 0.0), (134, 0.0), (138, 0.0), (162, 0.0), (294, 0.0), (431, 0.0), (579, 0.0)]
        
    outputs = [[words_list[i[0]],i[1]] for i in sorted_res[-10:]]
    # 按刚才的顺序取回最后10个句子和相似度
    # [['Apple iPhone 7 Plus (A1661) 128G 黑色 移动联通电信4G手机', 0.66742265521921251]
    
    
    return outputs  

## 测试

In [54]:
# words = '小米8 全面屏游戏智能手机 6GB+128GB 黑色 全网通4G 双卡双待  拍照手机'
# words = '荣耀 畅玩7X 4GB+32GB 全网通4G全面屏手机 标配版 铂光金'
words = 'Apple iPhone 8 Plus (A1864) 64GB 深空灰色 移动联通电信4G手机'
# words = '小米8'
# words = "黑色手机"
# words = 'Apple iPhone 8'
# words = '索尼 sony'
file_path = r'MobilePhoneTitle.txt'                     # 已经分词的文本
readed_path = r"MobilePhoneTitle_tfidf.txt"     # 已经分词的文本转成tiidf的词袋
outputs = main(words, file_path, readed_path)
# print(outputs)
for i in outputs[::-1]:                                           # 将句子逆序并打印出来
    print(i[0] + '     ' + str(i[1]))

Apple iPhone 8 Plus (A1864) 64GB 深空灰色 移动联通电信4G手机     1.0
Apple iPhone 8 (A1863) 64GB 深空灰色 移动联通电信4G手机     0.858115437204
Apple iPhone X (A1865) 64GB 深空灰色 移动联通电信4G手机     0.81818098341
Apple iPhone X (A1865) 64GB 深空灰色 移动联通电信4G手机     0.81818098341
Apple 苹果 iPhone 8 Plus移动联通电信4G手机 深空灰 官方标配 64GB     0.752617241065
苹果（Apple） iPhone 8 Plus  移动联通电信4G手机 深空灰 全网通(64GB)     0.725240200407
Apple 苹果8Plus  iPhone8 Plus 移动联通电信4G 手机 深空灰色 全网通 64GB     0.700648432367
Apple 苹果 iPhone X 全面屏4G手机 深空灰色 64GB     0.667423476215
Apple iPhone 7 Plus (A1661) 128G 黑色 移动联通电信4G手机     0.667422655219
Apple iPhone 7 Plus (A1661) 128G 黑色 移动联通电信4G手机     0.667422655219
